In [92]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "sejong"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"sejong":  {"iconservice": "https://sejong.net.solidwallet.io", "nid":83},
    "berlin":{"iconservice": "https://berlin.net.solidwallet.io", "nid":7},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [93]:
# Cell 1

from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os

ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
SEJONG_ORACLE = "cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [94]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [95]:
deployer_wallet = KeyWallet.load(bytes.fromhex("ca009dfacb4a0087167a7272038f2baf283c96d59c7a86eb5bf49481fbd155bd"))

print(icon_service.get_balance(deployer_wallet.get_address())/10**18)

print(deployer_wallet.get_address())

705236.2128410125
hx3f01840a599da07b0f620eeae7aa9c574169a4be


In [96]:
btest_wallet = KeyWallet.load(bytes.fromhex("ca009dfacb4a0087167a7272038f2baf283c96d59c7a86eb5bf49481fbd155bd"))

print(icon_service.get_balance(btest_wallet.get_address())/10**18)

print(btest_wallet.get_address())
print(btest_wallet.get_private_key())

705236.2128410125
hx3f01840a599da07b0f620eeae7aa9c574169a4be
ca009dfacb4a0087167a7272038f2baf283c96d59c7a86eb5bf49481fbd155bd


In [97]:
staking_wallet = KeyWallet.load(bytes.fromhex("ca009dfacb4a0087167a7272038f2baf283c96d59c7a86eb5bf49481fbd155bd"))

print(icon_service.get_balance(staking_wallet.get_address())/10**18)

print(staking_wallet.get_address())

705236.2128410125
hx3f01840a599da07b0f620eeae7aa9c574169a4be


In [98]:
# Cell 5a
## Testnet Sejong
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3228124be7c85e3e57edebf870c4075c33c34c5f'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx1eb5b209e4c6f95a958bddb322d76a2168e576b2'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx020042bb8742bfca138889e23817bee87ad8caa0'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx76d98fc79f779308d2e872b00e7571bebef59031'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx11781737f47520e3bfad0e132c5accb4e4d6fe4f'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx026ca90fdcf4851fa6c4e00403b3abe636a9f88b'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx4342802efce67d2441e69b062b1f0110b6a6f820'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cxd5df399205421930870e3b0798766aa31cffbe66'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxdc519895ef110220db2442ff1b3223182304b68a'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxf96a2e83808058fb9159824e324f36d9e91cfafa'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cxe6051c29d21e3eb4728cb1bf9f7ed521784e6acd'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cx0e706eca3552a6e607095319f4ad8cea37e779d4'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx041714d034919c8456d3606f8766f0169e35cb8e'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cxcc80a99118e2e35db8e9a575f86d87e33c2a9465'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx47f5be000e9893936e415b1b704001dd43467889'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cxb45058d398614a7c8cdf7be6f556fa0b39399799'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cxa2afb37647ae91ca6fbf35141b2ef2ac7105720a'}}

In [7]:
# new balanced address for UI test

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cxa8e5f6263b8403b24c935ee255e06be84cb45195'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx5ce1ed484edbf4332001ca40e5760acb64444a46'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxbde2a33bfadfad5703f076ec554453fcf21abc78'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cxb843d3a4935ca27911d3c1a2112095f512ff9d56'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxea58bbaabbad19f2f75a3aebcb97b088e3cbc9e9'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx9f782c71811566bff738263931b01bf6844ab8e3'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx9604fc2b833969260aea27373700e1547b69d34a'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cxb5e84abcd8ab014ca0a695fa5b3f23f7ff3ac138'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx386d9c545dfb74a0ee3441b05f011b75dbc9b7a9'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cx8c2cf0096636feb17bae2bb018b26dc87afb7bcf'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx692018cec4df32bed6b03922f1a3e42c7a684e0b'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cxf1ec0f7df577e30328e477eddbffca35f9acacac'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cxeed0ccc4ee5781abf65208178572e39fd6e49f53'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx7eb251b924e074d8a86f1a388db7aa7a3957b117'}}

In [89]:
# sejong reward test
contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx65c5a618e702d3cdf61d06eefc18a09051223445'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx85308dab5a08e1d89f8bc0411d4b6c15d5b96389'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cxebfafbcc62fda444786d20ce6b53b1b698f9ca6e'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx8dca2895370224748fce495b2ccd41bbe698bdf4'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cx146ff22d0d04b8bf2a17d18525bb2fb3fae3b5de'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cxbb5a6e4946ed4c763cda8e3ce295196b52564057'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx6d34cd2bfd5cf80647fa301530c9c687ea6eb4c4'},
 'rebalancing': {'zip': 'core_contracts/rebalancing.zip',
  'SCORE': 'cx698be4ae952c9c92a3bcef787a0bdd2a0f27a9e4'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx44e95ad5ab3661ead505d0ac40f3c8f51085e59b'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cx0999df3af49a227fc8799f0d78d19cd8fe4bb76d'},
 'router': {'zip': 'core_contracts/router.zip',
  'SCORE': 'cxc5e77c7faec7ae3098b526023b924f8dacc7fb77'},
    'feehandler': {'zip': 'core_contracts/feehandler.zip',
  'SCORE': 'cxxx'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxe4b51838552855c00d7f14b14b8f56b626a9e78c'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cxffcd3f9aedf58bcc07d83dd602dae8733407595d'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx4f5e21a99588dedaaac814316e17de9243102884'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx61943f594005f398942463316df655e010e5c1b8'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx8eb4032236bfa4b5f18141d55c5d7fee8ee46e71'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx35cd9d93a8ed19217a63244f0d8b0b201e7668b1'}}


In [103]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"core_contracts", "token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
    elif network == "sejong":
        contracts['oracle']['SCORE'] = SEJONG_ORACLE

                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 3000000000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    print(res)
    return res

def fast_send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()
    
    deploy = list(contracts.keys())[:]    
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        if 'bnXLM' in deploy:
            deploy.remove('bnXLM')
        if 'bnDOGE' in deploy:
            deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    
    res = deploy_SCORE(contracts['oracle'], {}, wallet, 0)
    results[f'{contracts["oracle"]}|deploy|{{}}'] = res
    oracle = res.get('scoreAddress', '')
    contracts['oracle']['SCORE'] = oracle
    
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    print(config)
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "hxfba37e91ccc13ec1dab115811f73e429cde44d48",  # ICX Station
            "hx231a795d1c719b9edf35c46b9daa4e0b5a1e83aa",  # iBriz-ICONOsphere
            "hxbc9c73670c79e8f6f8060551a792c2cf29a8c491",  # Mousebelt
            "hx28c08b299995a88756af64374e13db2240bc3142"}  # Parrot 9
    elif network == "sejong":
        preps = {
            "hx0f432ad57d1c851860a30b131c0843fb4f809650",  # ICX Station
            "hxf434a1b1f5270d4a028a4f737fa931dd70d7e2ec"}  # iBriz-ICONOsphere
    elif network == "berlin":
        preps = {
            "hxf526cc053c33a7c3a48b70111834cf3a71609f0c",  # ICX Station
            "hx38f35eff5e5516b48a713fe3c8031c94124191f0"}  # iBriz-ICONOsphere
        

    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate',
             'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Reading {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(btest_wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    if _print:
        print(result)
    return result


In [104]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

Deploying Balanced to sejong. Proceed (Yes/No)? Yes
core_contracts/governance.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Retrying in 4 seconds...
Status: 1

core_contracts/oracle.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

core_contracts/loans.zip
Retrying in 1 seconds...
Retrying in 2 seconds...


KeyboardInterrupt: 

In [79]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

['loans', 'staking', 'dividends', 'reserve', 'daofund', 'rewards', 'dex', 'rebalancing', 'governance', 'oracle', 'router', 'sicx', 'bnUSD', 'bnXLM', 'bnDOGE', 'baln', 'bwt']
------------------------------------------------------------------------------------------------------------------
Calling setAddresses, with parameters {'_addresses': {'loans': 'cx5133672576a06ce591f0ea87801913c928691e0c', 'staking': 'cx8f3fa876f8f496294d4e77c6bbf22586400c05cf', 'dividends': 'cx397c6eea82fcadf703ee22b9bb09d69d844a185d', 'reserve': 'cx4f4db9cc32027ac7e5f76142e34637965696faa8', 'daofund': 'cx63d6cd94e16964cbaba2ce1dfa648924fb6e1ba8', 'rewards': 'cx0f3e2dda33db2eba6e223bb25c6f6cdf6816e263', 'dex': 'cx0f6414c43084ff7581fc18b834eb93aabbd9e0c1', 'rebalancing': 'cxd99363dca42a611a88b69203cf87e2bd066b0e88', 'oracle': 'cx5f6d049dd10330dd2c24901c2f5090a1dddfc4a2', 'router': 'cx4e876517628912268bf028ffde1e8c0767e4991d', 'sicx': 'cxc45c52569af6d5ac49cf4af4be30df48799b058c', 'bnUSD': 'cx9b89df13545cadd36494e4a

In [80]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

------------------------------------------------------------------------------------------------------------------
Calling launchBalanced, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 0 **************************************************
Failure: {'code': 5, 'message': 'IllegalFormat'}
{'blockHash': '0x9a214d60f8d5e371ffb926de62ff1630152d6844d1039c785fd5cc322fcbf7cc', 'blockHeight': 338393, 'cumulativeStepUsed': 176535, 'eventLogs': [], 'failure': {'code': 5, 'message': 'IllegalFormat'}, 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [66]:
#create large number of users and send some icx to take loans.
for i in range(0,60):
    keywallet = KeyWallet.create()
    dict_wallets[str(keywallet.get_address())] = keywallet.get_private_key()
#     print(keywallet.get_address())
#     print(keywallet.get_private_key())
    send_icx(keywallet.get_address(), 501*10**18, btest_wallet)
    send_tx('loans', 300 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 50 * ICX}, keywallet)
dict_wallets

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': 

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6f

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6f

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6f

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6f

...

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': 

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': 

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6f

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 50000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x1043561a8829300000'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493'], 'data': ['0x1043561a8829300000', '0x4e6f6e65']} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6f

{'hxa486ff2efde16f018e6c27eadee33bcb1740a454': '511f6c0b2e38587afc40e693b75103636a4392da24a3e848ad9b263760bf127c',
 'hxde6b7e87adf06944468c382411a117ebb67dcad6': 'de25698ab332d94b2892ec7d3280f15e8849e13316f94f83c695e98e6489d890',
 'hxc1419a8ac7f22bf24226a27a365d85f3e0216f0e': 'a1c5a7966ae0eafb03cca39121e8e31efc3fc383e6be257835a78d2e83b0d005',
 'hx19ea14431ad1e3d0248aa5bcaa1a6e4ee7df75fa': 'd9121f0d3b9b5255b441716c964462ea54639b00aaae097253c4630ec6a96ee7',
 'hx782dc5cea2ed16020eda89e90584f0b8c9dcf14e': 'e58c647bf9521273b0f55cfc563e0ac365010663316f1d110a92b22ee6783c61',
 'hxdaa67efd333b7c416d062df858d889eed8b9f605': 'c2e2925657f8e49793c275f412dbf0cebee87cd6fec3b6e6b8e441aed89cb5a3',
 'hx22c0834a4e811c9e04490bb5a2916ac6270cd1e7': 'adc4e05fd39bee928fcf7d781bb39c3ea2d33984844f0697bbd6bca6f57ff5bb',
 'hxd66fdd4e139ce5c2041dc8410455b149942e1604': 'fbe249f1077e3ccfa1d0ad1a5ebcffa90724cf719c9de0a8b348c24a0af5ea30',
 'hx8748c3008d4c8537f85f743440d493c0ae9b3846': 'b99d63b2537936e057e43d7d76977b1c

In [67]:
def swap(from_token, to_token, amount):
    data = json.dumps({"method": "_swap", "params": {"toToken":str(to_token)}})
    params = {'_to': "cx4ab88c316656d83f40d847370825780592125cbb", '_value': amount, '_data': data.encode()}
    transaction = CallTransactionBuilder()\
        .from_(btest_wallet.get_address())\
        .to(from_token)\
        .step_limit(10000000)\
        .value(0)\
        .nid(NID)\
        .nonce(100)\
        .method("transfer")\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, btest_wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    get_tx_result(tx_hash)
    return tx_hash

swap('cxea120b827640e9d762d960476cd432babca8a97b','cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e',100*10**18)

Retrying in 1 seconds...


'0xefdaddc78ef1d22867b16ccb74fc4cc5758c7af09c89d695957dcd425639a00b'

In [69]:
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cxd9c1f035016dcc89bd16182ca6a45b43156a103b')\
    .value(0)\
    .step_limit(1000000000)\
    .nid(NID)\
    .nonce(100)\
    .method("rebalance")\
    .value(0)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(res)

Retrying in 1 seconds...
Retrying in 2 seconds...
{'blockHash': '0x74b208d3856280e4742a228d63089c34913f14de918dcc586775c46b17d3c2a3', 'blockHeight': 283168, 'cumulativeStepUsed': 15912810, 'eventLogs': [{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'cx4ab88c316656d83f40d847370825780592125cbb', '0x68ff1499526833ab'], 'data': ['0x7b226d6574686f64223a225f73776170222c22706172616d73223a7b22746f546f6b656e223a22637865613132306238323736343065396437363264393630343736636434333262616263613861393762227d7d']}, {'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx4ab88c316656d83f40d847370825780592125cbb', 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', '0x1edf307f4c65624bd'], 'data': ['0x4e6f6e65']}, {'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx4ab88c31665

In [102]:
# create preps in network

txs = []
start = 490
end = 530
for i in range(start, end):
    len_str = str(start)
    if len(len_str) == 2:
        j = "0" + str(i)
    else:
        j = str(i)
    params = {"name": "Test P-rep " + str(i), "country": "KOR", "city": "Unknown",
              "email": "nodehx9eec61296a7010c867ce24c20e69588e28321" + j + "@example.com", "website":
                  'https://nodehx9eec61296a7010c867ce24c20e69588e28321' + j + '.example.com',
              "details": 'https'
                         '://nodehx9eec61296a7010c867ce24c20e69588e28321' + j + '.example.com/details',
              "p2pEndpoint": "nodehx9eec61296a7010c867ce24c20e69588e28321" + j + ".example.com:7100",
              "nodeAddress": "hx9eec61296a7010c867ce24c20e69588e28321" + j}
    keywallet = KeyWallet.create()

    send_icx(keywallet.get_address(), 2100000000000000000000, btest_wallet)


    transaction = CallTransactionBuilder()\
        .from_(keywallet.get_address())\
        .to('cx0000000000000000000000000000000000000000')\
        .value(2000000000000000000000)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method('registerPRep')\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, keywallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(res)


Retrying in 1 seconds...
Retrying in 2 seconds...
{'blockHash': '0x15260b915d952539a77e95aff573fd054f320dc4bb1f3f1e790b845ffb9019cb', 'blockHeight': 1215201, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hx004c86b491ba7eb144ac436eff4d85e53b82d5a8'], 'data': ['0x6c6b935b8bbd400000', '0x43388f08df67b217ea021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hx004c86b491ba7eb144ac436eff4d85e53b82d5a8']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00

Retrying in 1 seconds...
{'blockHash': '0x2b62de44399ac41a1d9f4764b50f328c1e5d74b11902d82cbf918f7b1824c734', 'blockHeight': 1215215, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hx2f3c9cfa7ae2f417e02147db52aae53bbea31ffc'], 'data': ['0x6c6b935b8bbd400000', '0x43386d27415b166cc6021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hx2f3c9cfa7ae2f417e02147db52aae53bbea31ffc']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\

Retrying in 1 seconds...
{'blockHash': '0xfde381599ef1997453b1c91d0ae857b7fecd8ab7fd2b51008d8376bf9d25a3b2', 'blockHeight': 1215228, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hx4704676d6dcba56f107354eb9226c7f4374c4b2c'], 'data': ['0x6c6b935b8bbd400000', '0x43384b45a34e7ac1a2021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hx4704676d6dcba56f107354eb9226c7f4374c4b2c']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00

Retrying in 1 seconds...
Retrying in 2 seconds...
{'blockHash': '0xf8571e8f85640e622c295317302ef8cc83af484f247829068b986f3546907f19', 'blockHeight': 1215240, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hx0303f392dc0c353fc9225d098351746d170d2396'], 'data': ['0x6c6b935b8bbd400000', '0x433829640541df167e021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hx0303f392dc0c353fc9225d098351746d170d2396']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00

Retrying in 1 seconds...
Retrying in 2 seconds...
{'blockHash': '0x590cf8e470705711ab9db782d0d763a0defcae9afcace6782ac51437ce32543e', 'blockHeight': 1215253, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hx63c50a8b117cbe737448ec700640f3632dc3726a'], 'data': ['0x6c6b935b8bbd400000', '0x433807826735436b5a021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hx63c50a8b117cbe737448ec700640f3632dc3726a']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00

Retrying in 1 seconds...
{'blockHash': '0x79a2192748b9da59fbfed6d9e877dc381ded501658b9230adde0089da41f85cd', 'blockHeight': 1215267, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hx332a7cb5791d73b5469f19a246e18d84633356f2'], 'data': ['0x6c6b935b8bbd400000', '0x4337e5a0c928a7c036021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hx332a7cb5791d73b5469f19a246e18d84633356f2']}], 'logsBloom': b'\x00\x00\x00\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\

Retrying in 1 seconds...
{'blockHash': '0x13849c72d25c8a0df2d6be581f614fbe8a2107c34e6f1496026c3e62b0b80053', 'blockHeight': 1215280, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hxf0f78ab339a3b656cc8a082b06ad62cae9109f2f'], 'data': ['0x6c6b935b8bbd400000', '0x4337c3bf2b1c0c1512021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hxf0f78ab339a3b656cc8a082b06ad62cae9109f2f']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x00\x00\x00\x00\x00\x00\x00

Retrying in 1 seconds...
Retrying in 2 seconds...
{'blockHash': '0x1db398eb05d38d78dbe0a7138bb16011ed15ef64cff86639e90365b85eec28eb', 'blockHeight': 1215293, 'cumulativeStepUsed': 318400, 'eventLogs': [{'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['ICXBurnedV2(Address,int,int)', 'hxdc610a79c0ea0d8d9962a227494fc773c9b6f4a9'], 'data': ['0x6c6b935b8bbd400000', '0x4337a1dd8d0f7069ee021d1']}, {'scoreAddress': 'cx0000000000000000000000000000000000000000', 'indexed': ['PRepRegistered(Address)'], 'data': ['hxdc610a79c0ea0d8d9962a227494fc773c9b6f4a9']}], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08\x00\x0

In [84]:
# Cell 8
# Deploy or Update a single SCORE

contract_name = 'dex'
print(contracts[contract_name])
update = 1
params = {}
if update == 0 and contract_name != 'governance':
    params = {'_governance': contracts['governance']['SCORE']}

compress()
contract = contracts[contract_name]
if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'{"Updating" if update else "Deploying"} {contract_name} with params: {params} to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    res = deploy_SCORE(contract, params, btest_wallet, update)
print(res['scoreAddress'])

{'zip': 'core_contracts/dex.zip', 'SCORE': 'cx0f6414c43084ff7581fc18b834eb93aabbd9e0c1'}
Updating dex with params: {} to berlin. Proceed (Yes/No)? Yes
core_contracts/dex.zip
Retrying in 1 seconds...
Retrying in 2 seconds...
Status: 1

cx0f6414c43084ff7581fc18b834eb93aabbd9e0c1


In [64]:
send_tx('loans', 0 * ICX, 'depositAndBorrow', {'_asset': 'bnUSD', '_amount': 100 * ICX}, btest_wallet)

------------------------------------------------------------------------------------------------------------------
Calling depositAndBorrow, with parameters {'_asset': 'bnUSD', '_amount': 100000000000000000000} on the loans contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx6fd7d62b92f39033e01ca1ece14fd06ccf603903'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493', 'indexed': ['OriginateLoan(Address,str,int,str)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'bnUSD', '0x56bc75e2d63100000'], 'data': ['Loan of 100000000000000000000 bnUSD from Balanced.']} 

{'scoreAddress': 'cxea120b827640e9d762d960

{'blockHash': '0x2874c2680203331e1b7ce634719f84d6bd9136f69edf5bb6ce1a4850432a28c9',
 'blockHeight': 281656,
 'cumulativeStepUsed': 973005,
 'eventLogs': [{'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b',
   'indexed': ['OraclePrice(str,str,Address,int)',
    'USDICX',
    'BandChain',
    'cx6fd7d62b92f39033e01ca1ece14fd06ccf603903'],
   'data': ['0x84c5d90848544bb']},
  {'scoreAddress': 'cxae0fe2b1b4c3c224510b7168a2dd927791558493',
   'indexed': ['OriginateLoan(Address,str,int,str)',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be',
    'bnUSD',
    '0x56bc75e2d63100000'],
   'data': ['Loan of 100000000000000000000 bnUSD from Balanced.']},
  {'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b',
   'indexed': ['Mint(Address,int,bytes)',
    'hx3f01840a599da07b0f620eeae7aa9c574169a4be'],
   'data': ['0x56bc75e2d63100000', '0x4e6f6e65']},
  {'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx000

In [55]:

send_tx('governance', 0, 'setRebalancing', {'_address': contracts['rebalancing']['SCORE']}, btest_wallet)
send_tx('rebalancing', 0, 'setGovernance', {'_address': contracts['governance']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetSicx', {'_address': contracts['sicx']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetBnusd', {'_address': contracts['bnUSD']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetLoans', {'_address': contracts['loans']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'rebalancingSetDex', {'_address': contracts['dex']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'setLoansRebalance', {'_address': contracts['rebalancing']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'setLoansDex', {'_address': contracts['dex']['SCORE']}, btest_wallet)
send_tx('governance', 0, 'setRebalancingThreshold', {'_value': 5 * 10 ** 17}, btest_wallet)
send_tx('governance', 0, 'setMaxSellAmount', {"_sicx_value": 1000 *10 ** 18, "_bnusd_value": 1000 *10 ** 18}, btest_wallet)

------------------------------------------------------------------------------------------------------------------
Calling setRebalancing, with parameters {'_address': 'cxd9c1f035016dcc89bd16182ca6a45b43156a103b'} on the governance contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'blockHash': '0x7133ee5dec649ab689be6d20a93fb902ce86bcb756007fbe9dbc9ea977667f39', 'blockHeight': 281296, 'cumulativeStepUsed': 160520, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

Retrying in 1 seconds...
Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'blockHash': '0x9552c2d1be28be8f083bdd8f8210ab25caba437918a40fe49f9501b5a9a9a3b4', 'blockHeight': 281316, 'cumulativeStepUsed': 193170, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

{'blockHash': '0x533022fa2b8ca08a954360a2b89ea3b438c7fb88c86d30f76f6c9ad84458fa8d',
 'blockHeight': 281332,
 'cumulativeStepUsed': 148000,
 'eventLogs': [],
 'failure': {'code': 3, 'message': 'MethodNotFound'},
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [61]:
# create sicx/bnUSD market
send_tx('governance', 500*10**18, 'createBnusdMarket', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling createBnusdMarket, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------
Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx6fd7d62b92f39033e01ca1ece14fd06ccf603903'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cxa4d159170b6a20877d8976b91eb9968091819f35', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxa4d159170b6a20877d8976b91eb9968091819f35', 'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b', '0x3df454344fd9db6db'], 'data': []} 

{'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e', 'indexed': ['Mint(Address,int,bytes)', 'c

{'blockHash': '0x29f604a9a79eae410ed1b03e40577a97c9510021d76997cc3fdab75e12f63dff',
 'blockHeight': 281452,
 'cumulativeStepUsed': 16731255,
 'eventLogs': [{'scoreAddress': 'cxea120b827640e9d762d960476cd432babca8a97b',
   'indexed': ['OraclePrice(str,str,Address,int)',
    'USDICX',
    'BandChain',
    'cx6fd7d62b92f39033e01ca1ece14fd06ccf603903'],
   'data': ['0x84c5d90848544bb']},
  {'scoreAddress': 'cxa4d159170b6a20877d8976b91eb9968091819f35',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cxa4d159170b6a20877d8976b91eb9968091819f35',
    'cx24c14f9171beac581fa1b55efe9bb8946a3d4b5b',
    '0x3df454344fd9db6db'],
   'data': []},
  {'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e',
   'indexed': ['Mint(Address,int,bytes)',
    'cxa4d159170b6a20877d8976b91eb9968091819f35'],
   'data': ['0x3df454344fd9db6db', '0x4e6f6e65']},
  {'scoreAddress': 'cx3eb01d055d6f3c33eab8bd6db1b6ff6a605a843e',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx000000000000000000

In [17]:
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':4, '_name': 'BALN/sICX'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing : Executing'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing : Executing'}


Retrying in 2 seconds...
Status: 1


['sICX/ICX', 'sICX/bnUSD', 'BALN/bnUSD', 'BALN/sICX']


In [58]:

call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['rebalancing']['SCORE'])\
                    .method('getRebalancingStatus')\
                    .build()
result = icon_service.call(call)
print(result)

JSONRPCException: {'code': -30006, 'message': 'SystemError(-30006) : E0006:key is None'}

In [67]:
# add iusdc/usds as quote coin

send_tx('governance', 0, 'dexAddQuoteCoin', {'_address': 'cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183'}, btest_wallet)
send_tx('governance', 0, 'dexAddQuoteCoin', {'_address': 'cx2e008873de78f5bf82008a46b529d94924d9b6ae'}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling dexAddQuoteCoin, with parameters {'_address': 'cxb2c075e9130440dd64e6bfd3fe09e5c629f6e183'} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending : Pending'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing : Executing'}


Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'blockHash': '0x071d7d606775d8dfde107c461ab0730db423f33b8df807c33cb6f8bf0e64c544', 'blockHeight': 56178, 'cumulativeStepUsed': 175800, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending : Pending'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing : Executing'}


Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'blockHash': '0x816edd32941a9c54b69654a2a5506da0eb402a607d4ecf040d661f2669e422ab', 'blockHeight': 56182, 'cumulativeStepUsed': 175800, 'eventLogs': [], 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

{'blockHash': '0x816edd32941a9c54b69654a2a5506da0eb402a607d4ecf040d661f2669e422ab',
 'blockHeight': 56182,
 'cumulativeStepUsed': 175800,
 'eventLogs': [],
 'logsBloom': b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [15]:
call_tx('baln', "balanceOf", {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'})

------------------------------------------------------------------------------------------------------------------
Reading balanceOf, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the baln contract.
------------------------------------------------------------------------------------------------------------------
0x3507aa1d8ea84f3eab3


'0x3507aa1d8ea84f3eab3'

In [16]:
# governance contract needs to have baln to create the market

send_tx('governance', 0, 'createBalnMarket', {'_bnUSD_amount': 20*10**18, '_baln_amount': 10*10**18}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling createBalnMarket, with parameters {'_bnUSD_amount': 20000000000000000000, '_baln_amount': 10000000000000000000} on the governance contract.
------------------------------------------------------------------------------------------------------------------
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cxeed0ccc4ee5781abf65208178572e39fd6e49f53', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx9f782c71811566bff738263931b01bf6844ab8e3', 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289', '0x5af7359806ef09c154d'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx9f782c71811566bff738263931b01bf6844ab8e3', 'indexed': ['RewardsClaimed(Address,int)', 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289'], 'data': ['0x5af7359806ef09c154d']} 

{'scoreAddress': 'cxa8e5f6263b8403b24c935ee255e06be84cb45195',

{'blockHash': '0xc253ab56726be2c9eba2c5907822881bab5ad1ab4872db9081d5fac68fb99bd6',
 'blockHeight': 575800,
 'cumulativeStepUsed': 1312080,
 'eventLogs': [{'scoreAddress': 'cxeed0ccc4ee5781abf65208178572e39fd6e49f53',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'cx9f782c71811566bff738263931b01bf6844ab8e3',
    'cxdbe2036fd326eb04d9f1f336326eefb6ea558289',
    '0x5af7359806ef09c154d'],
   'data': ['0x4e6f6e65']},
  {'scoreAddress': 'cx9f782c71811566bff738263931b01bf6844ab8e3',
   'indexed': ['RewardsClaimed(Address,int)',
    'cxdbe2036fd326eb04d9f1f336326eefb6ea558289'],
   'data': ['0x5af7359806ef09c154d']},
  {'scoreAddress': 'cxa8e5f6263b8403b24c935ee255e06be84cb45195',
   'indexed': ['OriginateLoan(Address,str,int,str)',
    'cxdbe2036fd326eb04d9f1f336326eefb6ea558289',
    'bnUSD',
    '0x1158e460913d00000'],
   'data': ['Loan of 20000000000000000000 bnUSD from Balanced.']},
  {'scoreAddress': 'cx8c2cf0096636feb17bae2bb018b26dc87afb7bcf',
   'indexed': ['Mint(Address

In [14]:
send_tx('governance', 500*10**18, 'createBnusdMarket', {}, btest_wallet)


------------------------------------------------------------------------------------------------------------------
Calling createBnusdMarket, with parameters {} on the governance contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending : Pending'}


Retrying in 1 seconds...


Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing : Executing'}


Retrying in 2 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx8c2cf0096636feb17bae2bb018b26dc87afb7bcf', 'indexed': ['OraclePrice(str,str,Address,int)', 'USDICX', 'BandChain', 'cx386d9c545dfb74a0ee3441b05f011b75dbc9b7a9'], 'data': ['0x84c5d90848544bb']} 

{'scoreAddress': 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289', 'indexed': ['ICXTransfer(Address,Address,int)', 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289', 'cx5ce1ed484edbf4332001ca40e5760acb64444a46', '0x3df454344fd9db6db'], 'data': []} 

{'scoreAddress': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1', 'indexed': ['Mint(Address,int,bytes)', 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289'], 'data': ['0x3df454344fd9db6db', '0x4e6f6e65']} 

{'scoreAddress': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289', '0x3

{'blockHash': '0x17c20a0135fceeaaa337cfdfdcb95b2ac70fb4aab2306cff4af3dcf5a2c4f776',
 'blockHeight': 527143,
 'cumulativeStepUsed': 6394100,
 'eventLogs': [{'scoreAddress': 'cx8c2cf0096636feb17bae2bb018b26dc87afb7bcf',
   'indexed': ['OraclePrice(str,str,Address,int)',
    'USDICX',
    'BandChain',
    'cx386d9c545dfb74a0ee3441b05f011b75dbc9b7a9'],
   'data': ['0x84c5d90848544bb']},
  {'scoreAddress': 'cxdbe2036fd326eb04d9f1f336326eefb6ea558289',
   'indexed': ['ICXTransfer(Address,Address,int)',
    'cxdbe2036fd326eb04d9f1f336326eefb6ea558289',
    'cx5ce1ed484edbf4332001ca40e5760acb64444a46',
    '0x3df454344fd9db6db'],
   'data': []},
  {'scoreAddress': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1',
   'indexed': ['Mint(Address,int,bytes)',
    'cxdbe2036fd326eb04d9f1f336326eefb6ea558289'],
   'data': ['0x3df454344fd9db6db', '0x4e6f6e65']},
  {'scoreAddress': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1',
   'indexed': ['Transfer(Address,Address,int,bytes)',
    'hx0000000000000000000

In [18]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

['sICX/ICX', 'sICX/bnUSD', 'BALN/bnUSD', 'BALN/sICX']


In [ ]:
curl --location --request POST 'https://sejong.net.solidwallet.io/api/v3d' \
--header 'Content-Type: application/json' \
--data-raw '{
    "jsonrpc": "2.0",
    "method": "debug_getTrace",
    "id": 1234,
    "params": {
        "txHash": "0x7061e0883f4fb9ce7a53ade5fd41f77d8683a5c590c739d057210d76b2823954"
    }
}'

In [9]:
# transfer plain icx to sicx/icx pool
send_icx('hx177b10efe3018961a405cc7c20ee811c552831a5', 30000*10**18, btest_wallet)

'0x7dcda0069a30431ac9eee0f906aeb2c7ceaf24acff27a53604bd6eb74806a70b'

In [93]:
call = CallBuilder().from_(btest_wallet.get_address())\
                    .to('cx683c78174c740c3c6215de0c3ead1fad024324d6')\
                    .method('balanceOf')\
                    .params({'_owner': btest_wallet.get_address()})\
                    .build()
result = icon_service.call(call)
print(result)

0x113492426705a1a2d7800


In [100]:
call_tx('bnUSD', 'balanceOf', {'_owner': btest_wallet.get_address()})

------------------------------------------------------------------------------------------------------------------
Reading balanceOf, with parameters {'_owner': 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'} on the bnUSD contract.
------------------------------------------------------------------------------------------------------------------
0x202fdc94a07e0024092


'0x202fdc94a07e0024092'

In [94]:
# Cell 36 add omm

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 500000 * 10**18, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cx683c78174c740c3c6215de0c3ead1fad024324d6')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -31003, 'message': 'Executing : Executing'}


Retrying in 1 seconds...
Status: 1
{'scoreAddress': 'cx4342802efce67d2441e69b062b1f0110b6a6f820', 'indexed': ['Deposit(Address,Address,int)', 'cx683c78174c740c3c6215de0c3ead1fad024324d6', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x69e10de76676d0800000']} 

{'scoreAddress': 'cx683c78174c740c3c6215de0c3ead1fad024324d6', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cx4342802efce67d2441e69b062b1f0110b6a6f820', '0x69e10de76676d0800000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [13]:
# Add assets to existing pool
sICX_contribute_amount = int(10 * ICX)
bnUSD_contribute_amount = int(10 * ICX)
# bnUSD_amount = 80000 * ICX
# icx_price = 2

txns = [
#     {'contract': 'staking', 'value': 300 *10**18, 'method': 'stakeICX', 'params': {}},
#         {'contract': 'loans', 'value': 200000 *ICX, 'method': 'depositAndBorrow', 'params': {'_asset': 'bnUSD', '_amount': bnUSD_amount}},
#         {'contract': 'baln', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
#         {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': bnUSD_contribute_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': contracts['baln']['SCORE'], '_quoteToken': contracts['sicx']['SCORE'], '_baseValue': sICX_contribute_amount, '_quoteValue': bnUSD_contribute_amount}}
]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], btest_wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

------------------------------------------------------------------------------------------------------------------
Calling add, with parameters {'_baseToken': 'cxeed0ccc4ee5781abf65208178572e39fd6e49f53', '_quoteToken': 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1', '_baseValue': 10000000000000000000, '_quoteValue': 10000000000000000000} on the dex contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -31002, 'message': 'Pending : Pending'}


Retrying in 1 seconds...
************************************************** Status: 1 **************************************************
{'scoreAddress': 'cx9604fc2b833969260aea27373700e1547b69d34a', 'indexed': ['MarketAdded(int,Address,Address,int,int)', '0x4', 'cxeed0ccc4ee5781abf65208178572e39fd6e49f53', 'cxc79530e0e4081e2e898bac7baa0075fcf71e55c1'], 'data': ['0x8ac7230489e80000', '0x8ac7230489e80000']} 

{'scoreAddress': 'cx9604fc2b833969260aea27373700e1547b69d34a', 'indexed': ['Add(int,Address,int,int,int)', '0x4', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x8ac7230489e80000'], 'data': ['0x8ac7230489e80000', '0x8ac7230489e80000']} 

{'scoreAddress': 'cx9604fc2b833969260aea27373700e1547b69d34a', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cxf000000000000000000000000000000000000000', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x4', '0x8ac7230489e80000']} 

{'blockHash': '0x519451a5438d12ec6f2b68dff1